In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus
op = Optimus()

### Create a Spark Dataframe

In [109]:
# make some test data
columns = ['todo_id']
vals = [
    (1, ),
    (2, ),
    (3, ),
    (4, ),
    (5, )

]

# create DataFrame
df = op.spark.createDataFrame(vals, columns).repartition(1).cache()
df.table()

todo_id 1 (bigint)
1
2
3
4
5


### Create a pandas daraframe

In [126]:
import pandas as pd
data = {"todo_id": [6, 7, 8, 9,10], 
        "desc": ["bad", "good", "ugly", "tall","short"],
        "url": ["https://jsonplaceholder.typicode.com/todos/6", 
                "https://jsonplaceholder.typicode.com/todos/7", 
                "https://jsonplaceholder.typicode.com/todos/8", 
                "https://jsonplaceholder.typicode.com/todos/9",
                "https://jsonplaceholder.typicode.com/todos/10"]}
pdf =pd.DataFrame.from_dict(data)
pdf.head()

,todo_id,desc,url
0,6,bad,https://jsonplaceholder.typicode.com/todos/6
1,7,good,https://jsonplaceholder.typicode.com/todos/7
2,8,ugly,https://jsonplaceholder.typicode.com/todos/8
3,9,tall,https://jsonplaceholder.typicode.com/todos/9
4,10,short,https://jsonplaceholder.typicode.com/todos/10


In [101]:
from optimus.enricher import Enricher

e = Enricher("localhost",27017, "db","optimus")

### Send a Spark Dataframe

In [156]:
e.send(df_url)

### Send a Pandas Dataframe

In [153]:
e.send(pdf)

### Run the enrichment process

In [158]:
import requests 


def func_request(params):
    # You can use here whatever header or auth info you need to send. 
    # For mor information see the requests library
    url= "https://jsonplaceholder.typicode.com/todos/" + str(params["todo_id"])

    return requests.get(url)


def func_response(response):
    # Here you can parse de response
    return response["title"]


e.run(func_request= func_request, func_response= func_response)


No records available to process


In [159]:
e.save_to_csv("jazz.csv")


Removed 10 documents


### Merge with dataframe

In [28]:
e.flush()

Removed 10 documents


In [ ]:
### You can prepare a rul using nest

In [8]:
from pyspark.sql import functions as F
 
# Prepare the URL
df_url = df.cols.nest(("https://jsonplaceholder.typicode.com/todos/",F.col("todo_id")),"url")
df_url.table()

todo_id 1 (bigint),url 2 (string)
1,https://jsonplaceholder.typicode.com/todos/1
2,https://jsonplaceholder.typicode.com/todos/2
3,https://jsonplaceholder.typicode.com/todos/3
4,https://jsonplaceholder.typicode.com/todos/4
5,https://jsonplaceholder.typicode.com/todos/5


## Some other operations

In [38]:
e.show_keys()

['url', 'result', 'todo_id', '_id']

In [58]:
e.head("BOB",5)

Total documents:15
{'_id': ObjectId('5b940c37e254121740b5d19f'), 'todo_id': 1, 'url': 'https://jsonplaceholder.typicode.com/todos/1', 'result': 'delectus aut autem'}
{'_id': ObjectId('5b940c37e254121740b5d1a0'), 'todo_id': 2, 'url': 'https://jsonplaceholder.typicode.com/todos/2', 'result': 'quis ut nam facilis et officia qui'}
{'_id': ObjectId('5b940c37e254121740b5d1a1'), 'todo_id': 3, 'url': 'https://jsonplaceholder.typicode.com/todos/3', 'result': 'fugiat veniam minus'}
{'_id': ObjectId('5b940c37e254121740b5d1a2'), 'todo_id': 4, 'url': 'https://jsonplaceholder.typicode.com/todos/4', 'result': 'et porro tempora'}
{'_id': ObjectId('5b940c37e254121740b5d1a3'), 'todo_id': 5, 'url': 'https://jsonplaceholder.typicode.com/todos/5', 'result': 'laboriosam mollitia et enim quasi adipisci quia provident illum'}
